# GOP Tax Bill -- Analyzed

The GOP Tax Bill was notoriously known for being long and given on such short notice before the it was called to a vote. Although some provisions have changed, it is interesting to see if we can gather important information regarding the bill through a naive analysis of the sentence structure of the bill. In this analysis we aim to determine a conclusion regarding the bill and any interesting ideas that are encompassed in the bill.

Some of our implementations will include:
* Recording counts of bigrams or instances of two nouns together
* Reviewing the bill using unsupervised methods such as clustering
* Examining the bill from a network standpoint where nodes are nounds and edges are verbs


In [108]:
import PyPDF2
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [81]:
#import bill
bill = open('bill_text.pdf','rb')
billReader=PyPDF2.PdfFileReader(bill)

#do a check to make sure the parser worked somewhat
print(billReader.numPages)

#create a list to hold the string versions of the bill pages excluding table of contents
billPageList=[billReader.getPage(i).extractText() for i in range(6,billReader.numPages)]
billPageList=[re.split('(?<=[A-Za-z])\.',billPageList[i].replace('\n','').replace('‘‘','').replace('—','')) for i in range(len(billPageList))]

429


In [118]:
#Parsing of bill, cleaning basic things such as line numberings
billPageListParsed=[[re.sub(r'\(\w\)(.*)',r'\1',x) for x in page] for page in billPageList]
billPageListParsed=[[re.sub(r'\d+([A-Za-z]+)',r'\1',x) for x in page] for page in billPageListParsed]
billPageListParsed=[[re.sub(r'(.*)-\d+([a-z].*)',r'\1\2',x) for x in page] for page in billPageListParsed]
billPageListParsed=[[re.sub(r'(.*)-\d+(.*)',r'\1\2',x) for x in page] for page in billPageListParsed]
billPageListParsed=[[re.sub(r'(\D+)\d+ ([A-Za-z0-9]+)',r'\1 \2',x) for x in page] for page in billPageListParsed]
billPageListParsed=[[re.sub(r'(.*)VerDate.*',r'\1',x) for x in page][:-2] for page in billPageListParsed]
#flatten array
billPageListParsed=[y for x in billPageListParsed for y in x]

#Apparently need to keep document in string format not list-of-string format
for i in range(len(billPageListParsed)):
    if i == 0:
        continue
    elif len(billPageListParsed[i])==0:
        continue
    else:
        if re.search('[A-Z0-9]',billPageListParsed[i][0]):
            billPageListParsed[i-1]=billPageListParsed[i-1]+'.'
#billParsed=' '.join(billPageListParsed)

In [151]:
#count vectorize the bill such that we can find interesting insights
countVec = CountVectorizer(stop_words='english',max_df=0.01)
wordCount=countVec.fit_transform(billPageListParsed)
count=countVec.vocabulary_
count

{'percentbracket': 2878,
 '525': 617,
 'bracket': 1191,
 'threshold': 3682,
 '25': 347,
 'exceeds': 1914,
 'cent': 1270,
 '35': 454,
 'plus': 2949,
 '1339': 93,
 '39': 505,
 'bracketthresholdamounts': 1193,
 'percentbracketthreshold': 2879,
 'joint': 2387,
 'sur': 3591,
 'viving': 3928,
 'spouse': 3514,
 '90': 775,
 'head': 2179,
 'household': 2209,
 '67': 688,
 '500': 594,
 'estate': 1904,
 'trust': 3836,
 '550': 629,
 '260': 359,
 'married': 2526,
 'fil': 2047,
 'separate': 3383,
 '200': 251,
 '150': 138,
 'inflationadjustment': 2308,
 '2018': 267,
 'dollar': 1741,
 'reference': 3188,
 'creased': 1525,
 'to14': 3731,
 'multiplied': 2620,
 'cost': 1507,
 'living': 2478,
 'adjustment': 881,
 'stituting': 3544,
 'increase': 2269,
 'sen': 3381,
 'tence': 3631,
 'multiple': 2619,
 '100': 0,
 'rounded': 3340,
 'lowest': 2502,
 'livingadjustment': 2479,
 'ad': 867,
 'justment': 2396,
 'age': 901,
 'cpi': 1523,
 'normalized': 2699,
 'specialruleforadjustments': 3465,
 'withabaseyearafter2016

In [150]:
#count words associated with wealthy americans
wealthyDictionary={'estate':0,'trust':0,'corporation':0}
for word in count:
    if 'estate' in word.lower(): 
        wealthyDictionary['estate']+=count[word]
    if'trust' in word.lower(): 
        wealthyDictionary['trust']+=count[word]
    if 'corporation' in word.lower():
        wealthyDictionary['corporation']+=count[word]
wealthyDictionary
        

{'corporation': 52551, 'estate': 16745, 'trust': 27378}

In [154]:
#count words associated average americans
averageDictionary={'parent':0,'child':0,'dependent':0,'small business':0}
for word in count:
    if 'parent' in  word.lower():
        averageDictionary['parent']+=count[word]
    if 'child'  in word.lower() or 'children' in word.lower():
        averageDictionary['child']+=count[word]
    if 'dependent' in word.lower():
        averageDictionary['dependent']+=count[word]
    if 'small' in word.lower() and 'business' in word.lower():
        averageDictionary['small business']+=count[word]
averageDictionary

{'child': 25254, 'dependent': 22047, 'parent': 4892, 'small business': 3439}

In [155]:
#special industries
industryDictionary={'pharmaceuticals':0,'agriculture':0,'alternative_energy':0,'oil_and_gas':0}
for word in count:
    if 'pharma' in word.lower():
        industryDictionary['pharmaceuticals']=+count[word]
    if 'agri' in word.lower() or 'farm' in word.lower():
        industryDictionary['agriculture']+=count[word]
    if 'wind' in word.lower() or 'solar' in word.lower():
        industryDictionary['alternative_energy']+=count[word]
    if 'oil' in word.lower() or 'gas' in word.lower():
        industryDictionary['oil_and_gas']+=count[word]
industryDictionary

{'agriculture': 13522,
 'alternative_energy': 22978,
 'oil_and_gas': 4878,
 'pharmaceuticals': 0}

Some interesting notes about the bill already. The bill is currently in an imperfect parsed format where words can appear in a string of words with no spaces, i.e. 'hellomynameisbob', and 'hello', 'name' and 'bob' are all separate words. The words associated with wealthy or big corporations are all between $10^5$ and $10^6$ however, words associated with the average american don't come close to $10^5$